In [ ]:
# Let's try torch, the right py-kernel is python (myenv) and the env in conda is called ENVvae2, 
# more details can be found in the meeting notes with Lukas may 3


In [ ]:
# this is the tutorial: https://www.kaggle.com/code/schmiddey/variational-autoencoder-with-pytorch-vs-pca
# based on tabular data

In [1]:
import torch
print("PyTorch Version:", torch.__version__)

PyTorch Version: 2.0.0


In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
import sklearn as sk

In [8]:
df = pd.read_csv(r"/Users/karolinagustavsson/Code/Python_VAE/NH3.csv")

print(df)

         sampleID  year  wave  gender  age  annual_income  income_recode  \
0          1991_3  1991     0       1   21            NaN            2.0   
1          1991_4  1991     0       2   32            NaN            8.0   
2          1991_9  1991     0       2   48            NaN            8.0   
3         1991_10  1991     0       1   35            NaN            8.0   
4         1991_11  1991     0       1   48            NaN            6.0   
...           ...   ...   ...     ...  ...            ...            ...   
18820  1991_53593  1991     0       1   23            NaN            6.0   
18821  1991_53594  1991     0       2   26            NaN            6.0   
18822  1991_53595  1991     0       1   32            NaN            5.0   
18823  1991_53616  1991     0       1   85            NaN            3.0   
18824  1991_53618  1991     0       2   79            NaN            2.0   

       education  edu  ethnicity  ...  status  ucod_leading  diabetes  \
0            3

In [4]:
# Preparing the data

In [5]:
# apparently you have to restart and clear output 
# Define the columns of interest
columns_of_interest = ["sampleID", "status", "albumin", "alp", "lymph", "mcv",
                       "lncreat", "lncrp", "hba1c", "wbc", "rdw", "age"]

# Select only the necessary columns
df = df[columns_of_interest]

# Remove rows with any missing values
df = df.dropna()

# Keep the ID and status columns separate if needed later
ids = df['sampleID']
status = df['status']

# Remove these from the dataframe to be normalized
df = df.drop(['sampleID', 'status'], axis=1)

from sklearn.preprocessing import StandardScaler

# Initialize a scaler
scaler = StandardScaler()

# Normalize the data
data_normalized = scaler.fit_transform(df)

# Optionally convert back to DataFrame
df_normalized = pd.DataFrame(data_normalized, columns=df.columns)

from sklearn.model_selection import train_test_split

# Splitting data
x_train, x_test = train_test_split(df_normalized, test_size=0.2, random_state=123)
x_train, x_val = train_test_split(x_train, test_size=0.25, random_state=123)

# Check the shapes of the resulting data splits
print("Train data shape:", x_train.shape)
print("Validation data shape:", x_val.shape)
print("Test data shape:", x_test.shape)

print(df_normalized.describe())
print(x_train.head())

Train data shape: (9072, 10)
Validation data shape: (3025, 10)
Test data shape: (3025, 10)
            albumin           alp         lymph           mcv       lncreat  \
count  1.512200e+04  1.512200e+04  1.512200e+04  1.512200e+04  1.512200e+04   
mean  -3.458269e-16  1.202876e-16 -3.007191e-17  8.570493e-16  5.415292e-16   
std    1.000033e+00  1.000033e+00  1.000033e+00  1.000033e+00  1.000033e+00   
min   -5.163708e+00 -2.539327e+00 -3.230490e+00 -5.518384e+00 -4.379853e+00   
25%   -6.335773e-01 -6.675635e-01 -6.848479e-01 -5.169043e-01 -7.741436e-01   
50%   -1.006212e-01 -1.537462e-01 -6.702905e-02  5.840629e-02 -2.969089e-01   
75%    6.988140e-01  5.068761e-01  6.194364e-01  6.058776e-01  5.846864e-01   
max    5.228944e+00  6.525879e+00  4.526568e+00  5.217632e+00  6.217355e+00   

              lncrp         hba1c           wbc           rdw           age  
count  1.512200e+04  1.512200e+04  1.512200e+04  1.512200e+04  1.512200e+04  
mean  -2.556112e-16 -2.856831e-16  2.0674

In [9]:
### Wine notebook: https://www.kaggle.com/code/schmiddey/variational-autoencoder-with-pytorch-vs-pca

In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim
from torch.autograd import Variable
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt 

import pandas as pd
import numpy as np
from sklearn import preprocessing

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

########################## ändra
def load_data(path):
    # read in from csv
    df = pd.read_csv(DATA_PATH, sep=',', header=None, names=['Wine', 'Alcohol','Malic.acid','Ash','Acl',
                                                    'Mg', 'Phenols', 'Flavanoids','Nonflavanoid.phenols',
                                                    'Proanth','Color.int','Hue', 'OD','Proline'])
    # replace nan with -99
    df = df.fillna(-99)
    df_base = df.iloc[:, 1:]
    # get wine Label
    df_wine = df.iloc[:,0].values
    x = df_base.values.reshape(-1, df_base.shape[1]).astype('float32')
    # stadardize values
    standardizer = preprocessing.StandardScaler()
    x = standardizer.fit_transform(x)    
    return x, standardizer, df_wine

In [10]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

def load_data(path):
    # Load the dataset
    df = pd.read_csv(path)
    
    # Define the columns of interest
    columns_of_interest = ["sampleID", "status", "albumin", "alp", "lymph", "mcv",
                           "lncreat", "lncrp", "hba1c", "wbc", "rdw", "age"]
    
    # Select only the necessary columns
    df = df[columns_of_interest]
    
    # Remove rows with any missing values
    df = df.dropna()

    # Separate out IDs and status, and keep them for later use
    ids = df['sampleID']
    status = df['status']
    
    # Remove the ID and status columns from the dataframe to be normalized
    df_normalized = df.drop(['sampleID', 'status'], axis=1)
    
    # Normalize the data
    scaler = StandardScaler()
    x_normalized = scaler.fit_transform(df_normalized)
    
    # Convert normalized data back into a DataFrame with appropriate column names
    df_normalized = pd.DataFrame(x_normalized, columns=df_normalized.columns)
    
    # Return the normalized data and scaler, along with IDs and status for reintegration later
    return df_normalized, scaler, ids, status

# Example usage
DATA_PATH = '/Users/karolinagustavsson/Code/Python_VAE/NH3.csv'
x_train, standardizer, train_ids, train_status = load_data(DATA_PATH)


In [11]:
def numpyToTensor(x):
    x_train = torch.from_numpy(x).to(device)
    return x_train

from torch.utils.data import Dataset, DataLoader
class DataBuilder(Dataset):
    def __init__(self, path):
        self.x, self.standardizer, self.wine = load_data(DATA_PATH)
        self.x = numpyToTensor(self.x)
        self.len=self.x.shape[0]
    def __getitem__(self,index):      
        return self.x[index]
    def __len__(self):
        return self.len

In [12]:
import torch
from torch.utils.data import Dataset, DataLoader

class DataBuilder(Dataset):
    def __init__(self, path):
        # Load the data and ignore IDs and status for training purposes
        self.x, self.standardizer, self.ids, self.status = load_data(path)
        # Convert the dataframe to a PyTorch tensor
        self.x = torch.tensor(self.x.values, dtype=torch.float32)
        self.len = self.x.shape[0]

    def __getitem__(self, index):
        # Return the tensor at the specified index
        return self.x[index]

    def __len__(self):
        return self.len

In [13]:
data_set=DataBuilder(DATA_PATH)
trainloader=DataLoader(dataset=data_set,batch_size=1024)

In [14]:
type(trainloader.dataset.x)

torch.Tensor

In [15]:
data_set.x

tensor([[ 1.4982,  0.2133,  0.3334,  ..., -0.2466, -0.8054, -1.3980],
        [-0.1006,  0.1032, -0.4331,  ...,  2.5606, -0.0608, -0.8349],
        [-0.3671, -0.9979, -0.0613,  ..., -1.0555, -1.1033, -0.0159],
        ...,
        [ 1.7647,  0.9106, -1.2798,  ...,  0.3006,  0.1378, -1.2956],
        [-0.3671, -0.5942,  1.5805,  ..., -0.9603,  0.8328, -1.1421],
        [ 0.4323,  1.9749, -0.1586,  ..., -0.2942,  0.5349,  1.8782]])

In [16]:
### Build model and train it
class Autoencoder(nn.Module):
    def __init__(self,D_in,H=50,H2=12,latent_dim=3):
        
        #Encoder
        super(Autoencoder,self).__init__()
        self.linear1=nn.Linear(D_in,H)
        self.lin_bn1 = nn.BatchNorm1d(num_features=H)
        self.linear2=nn.Linear(H,H2)
        self.lin_bn2 = nn.BatchNorm1d(num_features=H2)
        self.linear3=nn.Linear(H2,H2)
        self.lin_bn3 = nn.BatchNorm1d(num_features=H2)
        
#         # Latent vectors mu and sigma
        self.fc1 = nn.Linear(H2, latent_dim)
        self.bn1 = nn.BatchNorm1d(num_features=latent_dim)
        self.fc21 = nn.Linear(latent_dim, latent_dim)
        self.fc22 = nn.Linear(latent_dim, latent_dim)

#         # Sampling vector
        self.fc3 = nn.Linear(latent_dim, latent_dim)
        self.fc_bn3 = nn.BatchNorm1d(latent_dim)
        self.fc4 = nn.Linear(latent_dim, H2)
        self.fc_bn4 = nn.BatchNorm1d(H2)
        
#         # Decoder
        self.linear4=nn.Linear(H2,H2)
        self.lin_bn4 = nn.BatchNorm1d(num_features=H2)
        self.linear5=nn.Linear(H2,H)
        self.lin_bn5 = nn.BatchNorm1d(num_features=H)
        self.linear6=nn.Linear(H,D_in)
        self.lin_bn6 = nn.BatchNorm1d(num_features=D_in)
        
        self.relu = nn.ReLU()
        
    def encode(self, x):
        lin1 = self.relu(self.lin_bn1(self.linear1(x)))
        lin2 = self.relu(self.lin_bn2(self.linear2(lin1)))
        lin3 = self.relu(self.lin_bn3(self.linear3(lin2)))

        fc1 = F.relu(self.bn1(self.fc1(lin3)))

        r1 = self.fc21(fc1)
        r2 = self.fc22(fc1)
        
        return r1, r2
    
    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu
        
    def decode(self, z):
        fc3 = self.relu(self.fc_bn3(self.fc3(z)))
        fc4 = self.relu(self.fc_bn4(self.fc4(fc3)))

        lin4 = self.relu(self.lin_bn4(self.linear4(fc4)))
        lin5 = self.relu(self.lin_bn5(self.linear5(lin4)))
        return self.lin_bn6(self.linear6(lin5))


        
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        # when the forward method is called, the decode method's output is the reconstructed batch (recon_batch), and 
        #the outputs mu and logvar are the parameters of the latent distribution used for sampling
        return self.decode(z), mu, logvar
    

In [17]:
class customLoss(nn.Module):
    def __init__(self):
        super(customLoss, self).__init__()
        self.mse_loss = nn.MSELoss(reduction="sum")
    
    # x_recon ist der im forward im Model erstellte recon_batch, x ist der originale x Batch, mu ist mu und logvar ist logvar 
    def forward(self, x_recon, x, mu, logvar):
        loss_MSE = self.mse_loss(x_recon, x)
        loss_KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        return loss_MSE + loss_KLD

In [18]:
# takes in a module and applies the specified weight initialization
def weights_init_uniform_rule(m):
    classname = m.__class__.__name__
    # for every Linear layer in a model..
    if classname.find('Linear') != -1:
        # get the number of the inputs
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.uniform_(-y, y)
        m.bias.data.fill_(0)

In [21]:
D_in = data_set.x.shape[1]
H = 50
H2 = 12
model = Autoencoder(D_in, H, H2).to(device)
model.apply(weights_init_uniform_rule)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [22]:
loss_mse = customLoss()

In [23]:
### Train model
epochs = 10
log_interval = 50
val_losses = []
train_losses = []

In [24]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(trainloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_mse(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
#        if batch_idx % log_interval == 0:
#            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                epoch, batch_idx * len(data), len(trainloader.dataset),
#                       100. * batch_idx / len(trainloader),
#                       loss.item() / len(data)))
    if epoch % 200 == 0:        
        print('====> Epoch: {} Average loss: {:.4f}'.format(
            epoch, train_loss / len(trainloader.dataset)))
        train_losses.append(train_loss / len(trainloader.dataset))

In [25]:
for epoch in range(1, epochs + 1):
    train(epoch)

In [26]:
### Evaluate
standardizer = trainloader.dataset.standardizer

In [27]:
model.eval()
test_loss = 0
# no_grad() bedeutet wir nehmen die vorher berechneten Gewichte und erneuern sie nicht
with torch.no_grad():
    for i, data in enumerate(trainloader):
        data = data.to(device)
        recon_batch, mu, logvar = model(data)

standardizer.inverse_transform(recon_batch[65].cpu().numpy())

In [32]:
standardizer.inverse_transform(recon_batch[65].cpu().numpy().reshape(1,-1))

array([[ 4.3136525 , 76.45169   , 31.511707  , 92.713585  ,  0.68965113,
         0.21598017,  5.0457606 ,  6.3450484 , 12.67099   , 58.881157  ]],
      dtype=float32)

standardizer.inverse_transform(data[5].cpu().numpy())

In [33]:
standardizer.inverse_transform(data[65].cpu().numpy().reshape(1, -1))

array([[ 4.1       , 55.        , 48.9       , 94.6       ,  0.51879376,
         0.19062035,  5.        ,  4.55      , 12.7       , 26.999998  ]],
      dtype=float32)

In [35]:
### Get embeddings / latent variables
mu_output = []
logvar_output = []

with torch.no_grad():
    for i, (data) in enumerate(trainloader):
            data = data.to(device)
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(data)

            
            mu_tensor = mu   
            mu_output.append(mu_tensor)
            mu_result = torch.cat(mu_output, dim=0)

            logvar_tensor = logvar   
            logvar_output.append(logvar_tensor)
            logvar_result = torch.cat(logvar_output, dim=0)

In [36]:
mu_result.shape

torch.Size([15122, 3])

In [37]:
mu_result[1:5,:]

tensor([[ 1.0218,  0.7471, -1.1497],
        [-0.1517, -0.1463,  0.1582],
        [-0.1189, -0.1455,  0.1300],
        [ 1.3931,  1.0089, -0.7016]])

In [ ]:
### Plot embeddings of VAE
#ax = plt.axes(projection='3d')


# Data for three-dimensional scattered points
#winetype = data_set.wine
#zdata = mu_result[:,0].cpu().numpy()
#xdata = mu_result[:,1].cpu().numpy()
#ydata = mu_result[:,2].cpu().numpy()
#ax.scatter3D(xdata, ydata, zdata, c=winetype);

So now it works but it is time to make a 2.0 version 